In [33]:
#Importing reuired libraries
import datetime as dt
import json
import pyspark
from pyspark.sql import functions as f

from pyspark.sql.functions import *

from pyspark.sql.types import *
from pyspark.sql import HiveContext,SparkSession


#Creating Spark Session 
spark = SparkSession.builder\
        .master("local")\
        .enableHiveSupport()\
        .appName("Spark JSON Functions")\
        .config("hive.exec.dynamic.partition", "true")\
        .config("hive.exec.dynamic.partition.mode", "nonstrict")\
        .config("spark.sql.warehouse.dir", "warehehousedir")\
        .getOrCreate()

In [25]:
df = spark.read.json("file:///home/abhijeet/Spark_UC/streaming_Asset/zipcode1.json", multiLine = "true")
#/home/abhijeet/Spark_UC/streaming_Asset/zipcode1.json
df.show()
print(df)


+--------------------+----+----+----+--------------------+-----+
|             batters|  id|name| ppu|             topping| type|
+--------------------+----+----+----+--------------------+-----+
|[[[1001, Regular]...|0001|Cake|0.55|[[5001, None], [5...|donut|
+--------------------+----+----+----+--------------------+-----+

DataFrame[batters: struct<batter:array<struct<id:string,type:string>>>, id: string, name: string, ppu: double, topping: array<struct<id:string,type:string>>, type: string]


# Convert "batters" Nested Structure to Simple DataFrame

In [32]:
sampleDF = df.withColumnRenamed("id", "key")
print(sampleDF)

DataFrame[batters: struct<batter:array<struct<id:string,type:string>>>, key: string, name: string, ppu: double, topping: array<struct<id:string,type:string>>, type: string]


In [30]:
batDF = sampleDF.select("key", "batters.batter")
batDF.printSchema()
batDF.show()



root
 |-- key: string (nullable = true)
 |-- batter: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)

+----+--------------------+
| key|              batter|
+----+--------------------+
|0001|[[1001, Regular],...|
+----+--------------------+



Creating a row for each element : explode

In [46]:
bat2DF = batDF.select("key", explode("batter").alias("new_batter"))
bat2DF.show()



+----+--------------------+
| key|          new_batter|
+----+--------------------+
|0001|     [1001, Regular]|
|0001|   [1002, Chocolate]|
|0001|   [1003, Blueberry]|
|0001|[1004, Devil's Food]|
+----+--------------------+



In [47]:
bat2DF.printSchema()

root
 |-- key: string (nullable = true)
 |-- new_batter: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)



In [48]:
bat2DF.select("key", "new_batter.*").show()



+----+----+------------+
| key|  id|        type|
+----+----+------------+
|0001|1001|     Regular|
|0001|1002|   Chocolate|
|0001|1003|   Blueberry|
|0001|1004|Devil's Food|
+----+----+------------+



Creating a row for each struct element : explode

In [49]:
finalBatDF = (sampleDF
        .select("key", explode("batters.batter").alias("new_batter"))
        .select("key", "new_batter.*")
        .withColumnRenamed("id", "bat_id")
        .withColumnRenamed("type", "bat_type"))
finalBatDF.show()



+----+------+------------+
| key|bat_id|    bat_type|
+----+------+------------+
|0001|  1001|     Regular|
|0001|  1002|   Chocolate|
|0001|  1003|   Blueberry|
|0001|  1004|Devil's Food|
+----+------+------------+



Select element from Array : topping

In [50]:
sampleDF.select(col("topping").getItem(0).id.alias("top_id"), col("topping").getItem(0).type.alias("top_type")).show()


+------+--------+
|top_id|top_type|
+------+--------+
|  5001|    None|
+------+--------+



# Convert "toppings" nested structure to simple DataFrame

Creating a row for each Array element : explode

In [51]:
topDF = (sampleDF
        .select("key", explode("topping").alias("new_topping"))
        .select("key", "new_topping.*")
        .withColumnRenamed("id", "top_id")
        .withColumnRenamed("type", "top_type")
        )
topDF.show(10, False)



+----+------+------------------------+
|key |top_id|top_type                |
+----+------+------------------------+
|0001|5001  |None                    |
|0001|5002  |Glazed                  |
|0001|5005  |Sugar                   |
|0001|5007  |Powdered Sugar          |
|0001|5006  |Chocolate with Sprinkles|
|0001|5003  |Chocolate               |
|0001|5004  |Maple                   |
+----+------+------------------------+

